In [2]:
from patient_info import *
from toolset.io_handler.file_error import *
from toolset.io_handler.meta_table_tool import *
import os

# global var
expression_root_dir = tcga_path + "/gene_expression_and_mutation/"
geneset_root_dir = proj_root + "/geneset_enrichment_db/"

#
os.getcwd()

'/Users/iris/Desktop/LCPF/toolset/case_handler'

In [12]:
# read table: one patient per row
assmbled_info = get_assembled_table()
assmbled_info.head(30)

,Unnamed: 0.1,Unnamed: 0,Cart,Case UUID,Case ID,Primary Site,Gender,Files,Seq,Exp,...,Program,Disease Type,Age at diagnosis,Days to death,Vital Status,Primary Diagnosis,Ethnicity,Race,RNA_Seq,WXS
0,0,0,Case Actions,a5f2576a-d628-47de-a5df-76a4f3105dcc,TCGA-L9-A743,Bronchus and lung,Male,69,6,4,...,TCGA,Adenomas and Adenocarcinomas,56 years 326 days,--,Alive,"Adenocarcinoma, NOS",not hispanic or latino,black or african american,51847dea-489f-4df9-84f8-26a6f2579620/35a9d225-...,4b20d50c-f569-4703-93ec-584c267e3fdf/0ef89e1c-...
1,1,1,Case Actions,c4c1469d-752f-42b6-8686-714ad4a8ac97,TCGA-75-6206,Bronchus and lung,Male,70,8,4,...,TCGA,Adenomas and Adenocarcinomas,--,--,Alive,"Adenocarcinoma, NOS",not reported,not reported,9123a775-d686-49ab-90e7-0dc572c2fcaf/7643fc1a-...,1ce62e4b-ba0b-4410-a508-569d44de1f4b/7bba1178-...
2,2,2,Case Actions,8d0736fe-261c-445c-bfd2-a3ea3ceaf367,TCGA-55-6971,Bronchus and lung,Female,79,9,6,...,TCGA,Adenomas and Adenocarcinomas,59 years 185 days,--,Alive,"Adenocarcinoma, NOS",not reported,white,c36f84e5-8c59-46fe-af32-7f70684383b2/07a66ae7-...,99eeb9be-e2d0-48e3-9940-daf1aee705e4/8383e3da-...
3,3,3,Case Actions,33c71ead-4c2a-476d-85ef-885dbbfc2cbd,TCGA-44-6774,Bronchus and lung,Female,75,9,4,...,TCGA,Adenomas and Adenocarcinomas,56 years 8 days,--,Alive,"Adenocarcinoma, NOS",not hispanic or latino,white,3cd897bd-893e-4497-b9a5-56141816cff4/525e9185-...,80f7616d-b100-45a1-92c5-e804690ccbd6/3fa65836-...
4,4,4,Case Actions,f77b6930-a1ce-446e-a15d-c018cbbecfee,TCGA-49-4510,Bronchus and lung,Female,79,10,4,...,TCGA,Adenomas and Adenocarcinomas,51 years 135 days,2 years 166 days,Dead,"Adenocarcinoma, NOS",not hispanic or latino,black or african american,951f2356-17f4-4f4f-a8ec-855c7e93dce1/7eab2708-...,0da50c40-ffa7-4bfc-8e87-02d456940500/285301af-...
5,5,5,Case Actions,e68219b0-a9c2-49df-8f0d-db5ea97fd2dc,TCGA-55-6543,Bronchus and lung,Female,71,8,4,...,TCGA,Adenomas and Adenocarcinomas,--,--,Alive,"Bronchio-alveolar carcinoma, mucinous",not reported,white,cb93a89c-f291-4042-a988-0fb1b392e4d7/4102816c-...,4b2e43e5-e9c7-478f-ac1c-cc5494b5cc8f/213c9ec5-...
6,6,6,Case Actions,05534064-8b43-4c82-95da-70fd83bd6560,TCGA-78-7540,Bronchus and lung,Female,73,7,6,...,TCGA,Adenomas and Adenocarcinomas,66 years 334 days,3 years 102 days,Dead,"Bronchio-alveolar carcinoma, mucinous",not reported,white,477d7181-e081-4577-ac06-e508b1239989/3d006e45-...,6f448407-401e-48d2-b31e-60e70e7c0889/52aa22b0-...
7,7,7,Case Actions,ab8d7f84-dd71-42c7-9471-4db967a9c89c,TCGA-50-6673,Bronchus and lung,Female,70,6,4,...,TCGA,Adenomas and Adenocarcinomas,84 years 8 days,22 days,Dead,"Adenocarcinoma, NOS",not reported,white,404b3699-4707-49aa-b0d2-a0999bbd48e6/2e86db3a-...,ce45d80b-ae25-4619-9f31-81183ad14a79/c981f20f-...
8,8,8,Case Actions,967d6548-5a84-4b7e-bc3f-2e522859fce6,TCGA-44-A47B,Bronchus and lung,Male,67,6,4,...,TCGA,Adenomas and Adenocarcinomas,79 years 15 days,--,Alive,"Adenocarcinoma, NOS",not hispanic or latino,white,ca34be5f-799c-4df4-af91-ec8a9e958d6c/fe770b01-...,9aed36b2-ad1b-489d-b953-f1788c85d637/97151b4e-...
9,9,9,Case Actions,fc79086c-af9a-4c27-b074-f7050b6f3381,TCGA-78-7155,Bronchus and lung,Male,71,8,4,...,TCGA,Adenomas and Adenocarcinomas,68 years 26 days,3 years 76 days,Dead,"Adenocarcinoma, NOS",not reported,white,d7439e19-2ab4-4209-9628-16e6ae6bc11e/962d56c6-...,3b0339db-235b-411b-8954-5504b38896b9/b7cbda3e-...


In [3]:
# prepare gene set dictionary
geneset_map = json_parser(geneset_root_dir + "c6.all.v2023.1.Hs.json")

def encode_expression(exp_df):
    # read expression data
    #print(exp_df)

    # empty list for gene sets: [[set1],[set2],...,[set200]]
    encoded_expression = []

    # for each gene set (pre-defined)
    for gene_set_index in geneset_map.keys():
        temp_container = []
        # for each gene in that gene set (the outer loop)
        for gene in geneset_map[gene_set_index]['geneSymbols']:
            # get the expression of that sigle gene
            try:
                temp_container.append(exp_df[exp_df["gene_name"] == gene]["tpm_unstranded"].values[0])
            except:
                temp_container.append(0)
        # append new set
        encoded_expression.append(temp_container)

    return encoded_expression



In [47]:
# Note: TPM (Transcripts Per Million) vs FPKM (Fragments Per Kilobase of transcript per Million mapped reads)
# 1. differ in their normalization strategies: FPKM is an older normalization method and was widely used before TPM became more popular.
# 2. TPM is a normalization method that calculates the expression level of a transcript in a sample relative to the total number of transcripts
# sequenced (in millions) in that sample
# 3. FPKM uses the total number of fragments instead of transcripts.

# input: patient information (a sigle row of a assembled file which contains info of all the patients)
def process_row(patient):
    # get basic info
    #print(patient)

    # get expression file path
    exp_dir = expression_root_dir + patient["RNA_Seq"]
    exist_flag, message = check_path_with_log(exp_dir)
    print(message) # log later

    # get content
    if exist_flag:
        expression_df = pd.read_csv(exp_dir, skiprows=1, sep="\t")
        expression_2d_arr = encode_expression(expression_df)


    return expression_2d_arr




In [48]:
# for every patient, test: only the first row assmbled_info[0:1]
# TODO: remove [0:1] below to iterate all cases

# struct: [  [[set1],[set2],...,[set200]],   [[set1],[set2],...,[set200]], ...,   [[set1],[set2],...,[set200]]  ]
# patient1: data[0]
# geneset1 of patient1: data[0][0]
# gene1 of geneset1 of patient1: data[0][0][0]

data = []
for index, patient in assmbled_info[0:5].iterrows():
    encoded = process_row(patient)
    data.append(encoded)



# check output
print(encoded)


VALID FILE: /Users/iris/Desktop/LCPF/tcga_db//gene_expression_and_mutation/51847dea-489f-4df9-84f8-26a6f2579620/35a9d225-0988-4e31-9a39-d54f3a63bea8.rna_seq.augmented_star_gene_counts.tsv
[[19.5982, 14.9256, 1.2475, 196.7105, 20.5692, 367.5455, 19.6302, 16.4955, 96.6584, 6.6543, 17.4972, 48.8504, 106.4916, 35.6033, 6.0813, 210.2699, 214.7559, 4.8686, 38.7535, 0.7971, 70.9591, 27.0292, 0.0701, 12.4487, 0.066, 3.6467, 0.0447, 8.348, 219.3436, 10.3763, 4.3565, 23.1804, 28.32, 108.0738, 1.0703, 123.1899, 29.3098, 6.0115, 0.4513, 19.9519, 11.9716, 31.6599, 8.6321, 19.5411, 28.2546, 1.99, 30.0209, 9.5446, 5.9989, 4.4252, 25.9936, 0.2634, 0.0205, 0.3457, 11.9994, 13.7986, 0.5069, 22.046, 1.8121, 116.4785, 13.4763, 22.8416, 27.9622, 8.3935, 0.049, 46.3687, 146.144, 2.5191, 58.5242, 14.182, 38.9546, 112.0102, 13.8262, 20.7465, 12.4775, 114.1795, 8.0963, 6.8409, 28.856, 8.5709, 117.2015, 52.4493, 44.4069, 10.6637, 17.9574, 5.9096, 36.5261, 10.7642, 3.8512, 0.3301, 13.812, 2.9066, 7.1948, 17.6782

In [1]:
encoded.shape()

NameError: name 'encoded' is not defined

In [4]:
print(geneset_map)

{'AKT_UP.V1_DN': {'systematicName': 'M2666', 'pmid': '15156201', 'exactSource': 'AKT_PLACEBO vs WT_PLACEBO; bottom 150 genes (diff. of means)', 'geneSymbols': ['ACKR3', 'ADGRL1', 'ADHFE1', 'ALPL', 'AMPD2', 'ANGPTL4', 'AQP5', 'ASF1A', 'ASS1', 'AXIN2', 'BCS1L', 'BEX4', 'BLVRB', 'BTBD2', 'C11orf71', 'C1R', 'C1S', 'CAP2', 'CAV2', 'CBS', 'CD248', 'CDC42BPA', 'CDH16', 'CDPF1', 'CELA1', 'CHAD', 'CHN2', 'CHST12', 'CNN2', 'COL17A1', 'CSDC2', 'DNAJA3', 'DPT', 'DRAM1', 'EDARADD', 'EFEMP1', 'EFEMP2', 'EIF4E3', 'ESR2', 'EXOSC1', 'FAH', 'FAM20C', 'FANK1', 'FAS', 'FBXO32', 'FHOD3', 'FKBP11', 'FMO2', 'FMO3', 'FUT10', 'FUT8', 'FXYD1', 'G6PC2', 'GALNT16', 'GAS1', 'GATAD2B', 'GHR', 'GNA14', 'GNMT', 'GRAMD1A', 'GSPT2', 'GSTA4', 'HEBP1', 'HIC1', 'HMGCS2', 'HPS1', 'HTRA1', 'IBSP', 'ID3', 'IFFO1', 'IFRD2', 'IL13RA1', 'IL17RC', 'IRS1', 'IRS2', 'ISLR', 'ITGBL1', 'ITIH5', 'ITPKB', 'KCNK3', 'KCNN4', 'KCTD12', 'KLF2', 'KRT15', 'KRT5', 'LIMD1', 'LPL', 'LTC4S', 'MATN2', 'MEPE', 'MERTK', 'MMAA', 'MOCOS', 'MRPS6', 'M

In [6]:
shape_record = []
for i in geneset_map.keys():
    shape_record.append(len(geneset_map[i]['geneSymbols']))

187
169
185
178
139
141
187
193
185
190
146
146
41
49
46
45
45
48
140
147
147
145
139
133
100
100
100
100
199
200
57
132
135
127
140
126
153
147
166
134
135
191
188
190
187
185
186
187
188
163
190
196
192
100
100
197
190
172
178
168
185
162
166
180
184
167
174
178
181
17
24
15
11
190
192
185
189
186
186
193
190
146
183
190
186
140
142
47
48
284
280
279
278
142
140
143
140
193
191
135
141
141
141
142
140
141
137
187
194
195
188
194
195
147
141
184
196
182
169
172
181
469
183
186
132
137
194
194
145
148
149
145
136
146
190
190
154
160
187
185
193
191
192
192
192
185
184
186
143
139
193
193
186
167
123
116
126
130
136
130
139
149
186
191
20
98
95
157
173
170
178
280
282
290
276
279
286
286
287
50
50
191
189
193
195
172
176
47
47


In [ ]:
# normalization: min-max - all patients, all genes